In [ ]:
import os
import re
import pickle
import random
import argparse
import logging
import numpy as np
import pandas as pd
from collections import Counter
from sklearn import set_config
from sklearn.multiclass import OneVsRestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import gensim
import string
import sys
from huggingface_hub import HfApi, HfFolder, upload_folder
from google.colab import userdata
import joblib

# Kiểm tra và cài đặt các thư viện cần thiết
try:
    import emoji
except ModuleNotFoundError:
    print("Installing emoji...")
    os.system("pip install emoji")
    import emoji
try:
    import wordninja
except ModuleNotFoundError:
    print("Installing wordninja...")
    os.system("pip install wordninja")
    import wordninja
try:
    import nlpaug.augmenter.word as naw
except ModuleNotFoundError:
    print("Installing nlpaug...")
    os.system("pip install nlpaug")
    import nlpaug.augmenter.word as naw
try:
    import gensim
except ModuleNotFoundError:
    print("Installing gensim...")
    os.system("pip install gensim")
    import gensim
try:
    import xgboost
except ModuleNotFoundError:
    print("Installing xgboost...")
    os.system("pip install xgboost")
    import xgboost

# Kiểm tra phiên bản scikit-learn
import sklearn
print(f"scikit-learn version: {sklearn.__version__}")
print(f"scikit-learn path: {sklearn.__file__}")
if sklearn.__version__ < '1.3':
    raise ImportError("scikit-learn version 1.3 or higher is required. Please upgrade using: !pip install --force-reinstall scikit-learn>=1.3")

# Thiết lập cấu hình scikit-learn
set_config(enable_metadata_routing=True)

# Thiết lập logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Định nghĩa các ánh xạ cho tiền xử lý
contraction_mapping = {
    "ain't": "is not", "aren't": "are not", "can't": "cannot", "'cause": "because",
    "could've": "could have", "couldn't": "could not", "didn't": "did not",
    "doesn't": "does not", "don't": "do not", "hadn't": "had not",
    "hasn't": "has not", "haven't": "have not", "he'd": "he would",
    "he'll": "he will", "he's": "he is", "how'd": "how did",
    "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
    "I'd": "I would", "I'd've": "I would have", "I'll": "I will",
    "I'll've": "I will have", "I'm": "I am", "I've": "I have",
    "i'd": "i would", "i'd've": "i would have", "i'll": "i will",
    "i'll've": "i will have", "i'm": "i am", "i've": "i have",
    "isn't": "is not", "it'd": "it would", "it'd've": "it would have",
    "it'll": "it will", "it'll've": "it will have", "it's": "it is",
    "let's": "let us", "ma'am": "madam", "mayn't": "may not",
    "might've": "might have", "mightn't": "might not",
    "mightn't've": "might not have", "must've": "must have",
    "mustn't": "must not", "mustn't've": "must not have",
    "needn't": "need not", "needn't've": "need not have",
    "o'clock": "of the clock", "oughtn't": "ought not",
    "oughtn't've": "ought not have", "shan't": "shall not",
    "sha'n't": "shall not", "shan't've": "shall not have",
    "she'd": "she would", "she'd've": "she would have",
    "she'll": "she will", "she'll've": "she will have", "she's": "she is",
    "should've": "should have", "shouldn't": "should not",
    "shouldn't've": "should not have", "so've": "so have", "so's": "so as",
    "this's": "this is", "that'd": "that would", "that'd've": "that would have",
    "that's": "that is", "there'd": "there would", "there'd've": "there would have",
    "there's": "there is", "here's": "here is", "they'd": "they would",
    "they'd've": "they would have", "they'll": "they will",
    "they'll've": "they will have", "they're": "they are",
    "they've": "they have", "to've": "to have", "wasn't": "was not",
    "we'd": "we would", "we'd've": "we would have", "we'll": "we will",
    "we'll've": "we will have", "we're": "we are", "we've": "we have",
    "weren't": "were not", "what'll": "what will", "what'll've": "what will have",
    "what're": "what are", "what's": "what is", "what've": "what have",
    "when's": "when is", "when've": "when have", "where'd": "where did",
    "where's": "where is", "where've": "where have", "who'll": "who will",
    "who'll've": "who will have", "who's": "who is", "who've": "who have",
    "why's": "why is", "why've": "why have", "will've": "will have",
    "won't": "will not", "won't've": "will not have", "would've": "would have",
    "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",
    "y'all'd": "you all would", "y'all'd've": "you all would have",
    "y'all're": "you all are", "y'all've": "you all have", "you'd": "you would",
    "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
    "you're": "you are", "you've": "you have", "u.s": "america", "e.g": "for example"
}

punct_mapping = {
    "‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ",
    "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"',
    '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha',
    '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi'
}

mispell_dict = {
    'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling',
    'counselling': 'counseling', 'theatre': 'theater', 'cancelled': "canceled", 'labour': 'labor',
    'organisation': "organization", 'wwii': 'world war 2', 'citicise': 'criticize', 'youtu ': 'youtube ',
    'Qoura': 'Quora', 'sallary': 'salary', 'Whta': 'What', 'narcisist': 'narcissist', 'howdo': 'how do',
    'whatare': 'what are', 'howcan': "how can", 'howmuch': 'how much', 'howmany': 'how many',
    'whydo': 'why do', 'doI': 'do I', 'theBest': 'the best', 'howdoes': 'how does',
    'mastrubation': 'masturbation', 'mastrubate': 'masturbate', 'mastrubating': 'masturbating',
    'pennis': 'penis', 'Etherium': 'Ethereum', 'narcissit': 'narcissist', 'bigdata': 'big data',
    '2k17': '2017', '2k18': '2018', 'qouta': 'quota', 'exboyfriend': 'ex boyfriend',
    'airhostess': 'air hostess', 'whst': 'what', 'watsapp': 'whatsapp',
    'demonitisation': 'demonetization', 'demonitization': 'demonetization',
    'demonetisation': "demonetization", 'pissed': 'angry', 'overthinking': 'overthinking'
}

punct_chars = list((set(string.punctuation) | {
    "’", "‘", "–", "—", "~", "|", "“", "”", "…", "'", "`", "_", "“"
}) - set(["#", "!", "?"]))
punct_chars.sort()
punctuation = "".join(punct_chars)
replace = re.compile("[%s]" % re.escape(punctuation))

# Hàm tải GloVe và emoji2vec
def load_embeddings(glove_path, emoji2vec_path, embedding_dim, logger):
    logger.info(f"Đang tải GloVe từ {glove_path}")
    glove_embeddings = {}
    try:
        with open(glove_path, 'r', encoding='utf-8') as f:
            for line in f:
                values = line.split()
                word = values[0]
                vector = np.asarray(values[1:], dtype='float32')
                glove_embeddings[word] = vector
        logger.info(f"Đã tải {len(glove_embeddings)} vector GloVe")
    except Exception as e:
        logger.error(f"Lỗi khi tải GloVe từ {glove_path}: {str(e)}")
        raise

    logger.info(f"Đang tải emoji2vec từ {emoji2vec_path}")
    emoji2vec = None
    try:
        with open(emoji2vec_path, 'r', encoding='utf-8') as f:
            first_line = f.readline()
            if not first_line.strip():
                raise ValueError(f"Tệp emoji2vec {emoji2vec_path} rỗng")
        emoji2vec = gensim.models.KeyedVectors.load_word2vec_format(
            emoji2vec_path, binary=False, unicode_errors='ignore'
        )
        logger.info(f"Đã tải {len(emoji2vec.key_to_index)} vector emoji")
    except Exception as e:
        logger.error(f"Lỗi khi tải emoji2vec từ {emoji2vec_path}: {str(e)}")
        raise
    return glove_embeddings, emoji2vec

# Hàm tạo đặc trưng từ văn bản với trọng số TF-IDF
def create_features(texts, tokenizer, glove_embeddings, emoji2vec, embedding_dim, max_len, logger):
    tfidf = TfidfVectorizer(max_features=30000)
    tfidf.fit(texts)
    tfidf_weights = tfidf.transform(texts).toarray()
    word_to_tfidf = {word: tfidf.idf_[idx] for word, idx in tfidf.vocabulary_.items()}

    sequences = tokenizer.texts_to_sequences(texts)
    padded = pad_sequences(sequences, maxlen=max_len, padding="post", truncating="post")
    features = np.zeros((len(texts), embedding_dim), dtype=np.float32)

    for i, seq in enumerate(padded):
        vectors = []
        weights = []
        for idx in seq:
            if idx == 0:  # Padding token
                continue
            word = tokenizer.index_word.get(idx, "<OOV>")
            weight = word_to_tfidf.get(word, 1.0)
            if emoji.is_emoji(word) and word in emoji2vec:
                vectors.append(emoji2vec[word])
                weights.append(weight)
            elif word in glove_embeddings:
                vectors.append(glove_embeddings[word])
                weights.append(weight)
        if vectors:
            weights = np.array(weights)
            vectors = np.array(vectors)
            weighted_avg = np.average(vectors, axis=0, weights=weights / np.sum(weights))
            features[i] = weighted_avg
        else:
            features[i] = np.zeros(embedding_dim)
    logger.info(f"Đã tạo đặc trưng cho {len(texts)} mẫu, kích thước: {features.shape}")
    return features

# Hàm tiền xử lý văn bản
def clean_text(text, logger):
    if not isinstance(text, str) or not text:
        return ""
    logger.debug(f"Văn bản gốc: {text}")

    for contraction, full_form in contraction_mapping.items():
        text = text.replace(contraction, full_form)

    for p, replacement in punct_mapping.items():
        text = text.replace(p, replacement)

    def split_hashtag(match):
        hashtag = match.group(0)[1:]
        words = wordninja.split(hashtag)
        return ' '.join(words)
    text = re.sub(r"#\w+", split_hashtag, text)

    text = re.sub(r"http\S*|\S*\.com\S*|\S*www\S*", " ", text)
    text = re.sub(r"\s@\S+", " ", text)

    text = replace.sub(" ", text)

    text = text.lower()
    words = text.split()
    words = [mispell_dict.get(word, word) for word in words]
    text = ' '.join(words)
    text = re.sub(r"\s+", " ", text).strip()

    logger.debug(f"Văn bản đã xử lý: {text}")
    return text

# Lớp xử lý dữ liệu GoEmotions
class GoemotionsProcessor:
    def __init__(self, args, logger):
        self.args = args
        self.logger = logger

    def get_labels(self):
        label_file = os.path.join(self.args.data_dir, self.args.label_file)
        self.logger.info(f"Đang đọc tệp nhãn tại: {label_file}")
        if not os.path.exists(label_file):
            self.logger.warning(f"Không tìm thấy tệp nhãn tại {label_file}. Sử dụng nhãn mặc định GoEmotions.")
            labels = [
                'admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion',
                'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment',
                'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism',
                'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral'
            ]
            self.logger.info(f"Sử dụng {len(labels)} nhãn mặc định")
            return labels
        try:
            with open(label_file, "r", encoding="utf-8") as f:
                labels = [line.strip() for line in f if line.strip()]
            if not labels:
                self.logger.error(f"Tệp nhãn {label_file} rỗng")
                raise ValueError(f"Tệp nhãn {label_file} rỗng")
            self.logger.info(f"Đã đọc {len(labels)} nhãn từ {label_file}")
            return labels
        except Exception as e:
            self.logger.error(f"Lỗi khi đọc tệp nhãn {label_file}: {str(e)}")
            raise

    def _read_file(self, input_file):
        if not os.path.exists(input_file):
            self.logger.error(f"Không tìm thấy tệp dữ liệu tại {input_file}")
            raise FileNotFoundError(f"Không tìm thấy tệp dữ liệu tại {input_file}")
        try:
            df = pd.read_csv(input_file, sep='\t', header=None, names=['text', 'labels', 'id'])
            self.logger.info(f"Đã đọc {len(df)} dòng từ {input_file}")
            if df.empty:
                self.logger.error(f"Tệp dữ liệu {input_file} rỗng")
                raise ValueError(f"Tệp dữ liệu {input_file} rỗng")
            return df
        except Exception as e:
            self.logger.error(f"Lỗi khi đọc tệp {input_file}: {str(e)}")
            raise

    def _augment_data(self, texts, labels):
        aug = naw.SynonymAug(aug_p=0.3)
        augmented_texts, augmented_labels = [], []
        for text, label in zip(texts, labels):
            augmented_texts.append(text)
            augmented_labels.append(label)
            aug_text = aug.augment(text)[0]
            augmented_texts.append(aug_text)
            augmented_labels.append(label)
        self.logger.info(f"Số mẫu sau tăng cường: {len(augmented_texts)}")
        return augmented_texts, augmented_labels

    def _balance_labels(self, examples, label_list_len, set_type):
        if set_type != "train":
            return examples
        self.logger.info("Cân bằng nhãn cho tập huấn luyện")
        label_counts = Counter()
        for ex in examples:
            label_counts.update(ex['labels'])
        self.logger.info(f"Phân bố nhãn ban đầu: {dict(label_counts)}")
        counts = [count for count in label_counts.values() if count > 0]
        target_count = min(int(np.median(counts) * 6.0), len(examples) // 2)
        self.logger.info(f"Số lượng mục tiêu mỗi nhãn: {target_count}")
        balanced_examples = []
        for label in range(label_list_len):
            samples_with_label = [ex for ex in examples if label in ex['labels']]
            current_count = label_counts.get(label, 0)
            if current_count == 0:
                continue
            elif current_count < target_count:
                samples_needed = target_count - current_count
                oversampled = random.choices(samples_with_label, k=samples_needed)
                balanced_examples.extend(oversampled)
            else:
                samples_to_keep = max(target_count, int(current_count * 0.5))
                balanced_examples.extend(random.sample(samples_with_label, min(samples_to_keep, len(samples_with_label))))
        balanced_examples.extend([ex for ex in examples if not any(label in ex['labels'] for label in range(label_list_len))])
        random.shuffle(balanced_examples)
        new_label_counts = Counter()
        for ex in balanced_examples:
            new_label_counts.update(ex['labels'])
        self.logger.info(f"Phân bố nhãn sau cân bằng: {dict(new_label_counts)}")
        self.logger.info(f"Số mẫu ban đầu: {len(examples)}, Số mẫu sau cân bằng: {len(balanced_examples)}")
        return balanced_examples

    def get_examples(self, mode):
        file_map = {
            'train': self.args.train_file,
            'dev': self.args.dev_file,
            'test': self.args.test_file
        }
        file_to_read = file_map.get(mode)
        if not file_to_read:
            raise ValueError("Mode phải là 'train', 'dev', hoặc 'test'")
        file_path = os.path.join(self.args.data_dir, file_to_read)
        self.logger.info(f"Đang đọc dữ liệu {mode} từ {file_path}")
        df = self._read_file(file_path)
        return self._create_examples(df, mode)

    def _create_examples(self, df, set_type):
        examples = []
        label_list_len = len(self.get_labels())
        label_counts = Counter()
        for i, row in df.iterrows():
            guid = f"{set_type}-{i}"
            raw_text = row['text']
            label_str = str(row['labels'])
            try:
                label = [int(l) for l in label_str.split(',') if l.strip().isdigit()]
                label = [l for l in label if 0 <= l < label_list_len]
                if not label:
                    self.logger.warning(f"Không có nhãn hợp lệ tại dòng {i}: {label_str}. Bỏ qua.")
                    continue
                label_counts.update(label)
            except (ValueError, IndexError) as e:
                self.logger.warning(f"Nhãn không hợp lệ tại dòng {i}: {label_str}. Bỏ qua. Lỗi: {e}")
                continue
            cleaned_text = clean_text(raw_text, self.logger)
            examples.append({
                'guid': guid,
                'text': cleaned_text,
                'labels': label
            })
        self.logger.info(f"Đã tạo {len(examples)} mẫu từ {set_type}")
        self.logger.info(f"Phân bố nhãn cho {set_type} (trước cân bằng): {dict(label_counts)}")
        if not examples:
            self.logger.error(f"Không tạo được mẫu từ {set_type}. Kiểm tra tệp dữ liệu!")
            raise ValueError(f"Không tạo được mẫu từ {set_type}")
        examples = self._balance_labels(examples, label_list_len, set_type)
        return examples

# Hàm tải dữ liệu
def load_data(args, logger):
    processor = GoemotionsProcessor(args, logger)
    label_list = processor.get_labels()
    num_labels = len(label_list)
    def load_and_cache(mode):
        cached_file = os.path.join(args.data_dir, f"cached_{mode}_data.pkl")
        if os.path.exists(cached_file):
            logger.info(f"Đang tải dữ liệu đã cache từ {cached_file}")
            with open(cached_file, 'rb') as f:
                data = pickle.load(f)
            return data['texts'], data['labels']
        logger.info(f"Đang xử lý dữ liệu {mode}")
        examples = processor.get_examples(mode)
        texts = [ex['text'] for ex in examples]
        labels = [ex['labels'] for ex in examples]
        if mode == 'train':
            texts, labels = processor._augment_data(texts, labels)
        with open(cached_file, 'wb') as f:
            pickle.dump({'texts': texts, 'labels': labels}, f)
        logger.info(f"Đã cache dữ liệu {mode} vào {cached_file}")
        return texts, labels
    try:
        train_texts, train_labels = load_and_cache('train')
        val_texts, val_labels = load_and_cache('dev')
        test_texts, test_labels = load_and_cache('test')
        logger.info(f"Số mẫu huấn luyện: {len(train_texts)}, Số mẫu xác thực: {len(val_texts)}, Số mẫu kiểm tra: {len(test_texts)}")
    except Exception as e:
        logger.error(f"Lỗi khi tải dữ liệu: {str(e)}")
        raise
    return train_texts, val_texts, test_texts, train_labels, val_labels, test_labels, label_list

# Hàm mã hóa nhãn thành dạng multi-hot
def to_multi_hot(label_lists, num_labels):
    m = np.zeros((len(label_lists), num_labels), dtype=np.int32)
    for i, labs in enumerate(label_lists):
        m[i, labs] = 1
    return m

# Hàm tính trọng số lớp
def compute_class_weights(labels, num_labels, logger):
    label_counts = np.zeros(num_labels)
    for labs in labels:
        for l in labs:
            label_counts[l] += 1

    epsilon = 1e-8
    label_counts = np.maximum(label_counts, epsilon)

    total_samples = len(labels)
    class_weights = {}
    for i in range(num_labels):
        if label_counts[i] > 0:
            class_weights[i] = total_samples / (num_labels * label_counts[i])
            class_weights[i] = max(class_weights[i], 1.0)
        else:
            class_weights[i] = 1.0

    logger.info(f"Phân bố nhãn: {label_counts}")
    logger.info(f"Trọng số lớp: {class_weights}")
    return class_weights

# Hàm tìm ngưỡng tối ưu trên tập xác thực
def find_optimal_thresholds(model, X_val, y_val, num_labels, logger):
    pred_probs = model.predict_proba(X_val)
    thresholds = np.arange(0.1, 0.5, 0.05)
    best_thresholds = np.ones(num_labels) * 0.25  # Giá trị mặc định
    best_f1 = 0.0

    for threshold in thresholds:
        y_pred = (pred_probs >= threshold).astype(int)
        micro_f1 = f1_score(y_val, y_pred, average='micro', zero_division=0)
        if micro_f1 > best_f1:
            best_f1 = micro_f1
            best_thresholds = np.ones(num_labels) * threshold

    logger.info(f"Ngưỡng tối ưu: {best_thresholds}, Micro F1: {best_f1:.4f}")
    return best_thresholds

# Hàm dự đoán cho các ví dụ mới
def predict_examples(model, tokenizer, examples, label_list, glove_embeddings, emoji2vec, embedding_dim, max_len, thresholds, logger):
    print("\n===== Dự đoán cho các ví dụ kiểm tra mới =====")
    if not examples:
        print("Không có ví dụ kiểm tra nào được cung cấp!")
        return []
    cleaned_examples = [clean_text(ex, logger) for ex in examples]
    print("Văn bản sau khi tiền xử lý:", cleaned_examples)
    features = create_features(cleaned_examples, tokenizer, glove_embeddings, emoji2vec, embedding_dim, max_len, logger)
    pred_probs = model.predict_proba(features)
    predictions = np.zeros_like(pred_probs, dtype=int)
    for i in range(len(label_list)):
        predictions[:, i] = (pred_probs[:, i] >= thresholds[i]).astype(int)

    results = []
    for i, example in enumerate(examples):
        predicted_labels = [label_list[j] for j in range(len(label_list)) if predictions[i][j] == 1]
        top_indices = np.argsort(pred_probs[i])[-3:][::-1]
        top_labels = [label_list[j] for j in top_indices]
        top_probs = [pred_probs[i][j] for j in top_indices]
        print(f"\nVí dụ {i+1}: {example}")
        print(f"Dự đoán (ngưỡng tùy chỉnh): {predicted_labels if predicted_labels else 'Không có nhãn'}")
        print(f"Top-3 nhãn: {list(zip(top_labels, top_probs))}")
        results.append({
            "text": example,
            "labels": predicted_labels if predicted_labels else "Không có nhãn",
            "top_labels": list(zip(top_labels, top_probs)),
            "probs": pred_probs[i].tolist()
        })
    return results

# Hàm chính
def main():
    # Cấu hình tham số
    parser = argparse.ArgumentParser(description="XGBoost Multi-Label Classification for GoEmotions or VAAFI")
    parser.add_argument("--data_dir", default="/content/drive/MyDrive/Goemotions/data", type=str, help="Thư mục chứa dữ liệu")
    parser.add_argument("--train_file", default="train.tsv", type=str, help="Tệp dữ liệu huấn luyện")
    parser.add_argument("--dev_file", default="dev.tsv", type=str, help="Tệp dữ liệu xác thực")
    parser.add_argument("--test_file", default="test.tsv", type=str, help="Tệp dữ liệu kiểm tra")
    parser.add_argument("--label_file", default="labels.txt", type=str, help="Tệp chứa danh sách nhãn")
    parser.add_argument("--glove_path", default="/content/drive/MyDrive/Goemotions/glove.6B.300d.txt", type=str, help="Đường dẫn đến tệp GloVe")
    parser.add_argument("--emoji2vec_path", default="/content/drive/MyDrive/Goemotions/emoji2vec.txt", type=str, help="Đường dẫn đến tệp emoji2vec")
    parser.add_argument("--ckpt_dir", default="/content/drive/MyDrive/Goemotions/checkpoints", type=str, help="Thư mục lưu checkpoint mô hình")
    parser.add_argument("--model_type", default="goemotions-xgb", type=str, help="Loại mô hình (goemotions-xgb hoặc vaafi-xgb)")
    parser.add_argument("--hf_repo_id", default="Songnguyen263/{model_type}", type=str, help="ID kho lưu trữ Hugging Face (username/repo_name), dùng {model_type} để thay thế")

    args = parser.parse_args([arg for arg in sys.argv[1:] if not arg.startswith('-f') and not arg.endswith('.json')])

    # Thay thế {model_type} trong hf_repo_id
    args.hf_repo_id = args.hf_repo_id.format(model_type=args.model_type)

    # Mount Google Drive
    try:
        from google.colab import drive
        drive.mount('/content/drive', force_remount=True)
        print("Google Drive mounted successfully")
    except Exception as e:
        logger.error(f"Error mounting Google Drive: {str(e)}")
        raise

    # Tạo thư mục checkpoint
    try:
        os.makedirs(args.ckpt_dir, exist_ok=True)
        logger.info(f"Thư mục checkpoint: {args.ckpt_dir}")
    except Exception as e:
        logger.error(f"Lỗi khi tạo thư mục checkpoint: {str(e)}")
        raise

    # Tải dữ liệu
    train_texts, val_texts, test_texts, train_labels, val_labels, test_labels, label_list = load_data(args, logger)
    num_labels = len(label_list)

    # Mã hóa nhãn
    y_train = to_multi_hot(train_labels, num_labels)
    y_val = to_multi_hot(val_labels, num_labels)
    y_test = to_multi_hot(test_labels, num_labels)

    # Token hóa và tạo đặc trưng
    vocab_size = 30000
    max_len = 100
    embedding_dim = 300

    tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>", filters='!"$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
    tokenizer.fit_on_texts(train_texts + ['pissed', 'overthinking'])  # Thêm từ khóa quan trọng

    try:
        with open(os.path.join(args.ckpt_dir, f"{args.model_type}_tokenizer.pkl"), "wb") as f:
            pickle.dump(tokenizer, f)
        logger.info(f"Đã lưu tokenizer vào {args.model_type}_tokenizer.pkl")
    except Exception as e:
        logger.error(f"Lỗi khi lưu tokenizer: {str(e)}")
        raise
    logger.info(f"Từ điển tokenizer: {dict(list(tokenizer.word_index.items())[:50])}")
    logger.info(f"'angry' trong từ điển: {'angry' in tokenizer.word_index}")
    logger.info(f"'overthinking' trong từ điển: {'overthinking' in tokenizer.word_index}")

    # Tải và tạo đặc trưng từ embeddings
    glove_embeddings, emoji2vec = load_embeddings(args.glove_path, args.emoji2vec_path, embedding_dim, logger)
    X_train = create_features(train_texts, tokenizer, glove_embeddings, emoji2vec, embedding_dim, max_len, logger)
    X_val = create_features(val_texts, tokenizer, glove_embeddings, emoji2vec, embedding_dim, max_len, logger)
    X_test = create_features(test_texts, tokenizer, glove_embeddings, emoji2vec, embedding_dim, max_len, logger)

    # Xây dựng mô hình XGBoost
    class_weights = compute_class_weights(train_labels, num_labels, logger)
    model = OneVsRestClassifier(
        XGBClassifier(
            max_depth=6,
            learning_rate=0.1,
            n_estimators=200,  # Tăng số lượng estimators
            objective='binary:logistic',
            eval_metric='logloss',
            random_state=42,
            n_jobs=-1
        )
    )

    # Huấn luyện mô hình với early stopping
    try:
        logger.info("Đang huấn luyện mô hình XGBoost với early stopping...")
        model.fit(
            X_train, y_train,
            estimator__eval_set=[(X_val, y_val)],
            estimator__early_stopping_rounds=10,
            estimator__verbose=False
        )
    except Exception as e:
        logger.warning(f"Early stopping không khả dụng: {str(e)}. Sử dụng fit thông thường.")
        model.fit(X_train, y_train)
    logger.info("Huấn luyện hoàn tất")

    # Tìm ngưỡng tối ưu trên tập xác thực
    thresholds = find_optimal_thresholds(model, X_val, y_val, num_labels, logger)

    # Đánh giá mô hình
    pred_probs = model.predict_proba(X_test)
    y_pred = np.zeros_like(pred_probs, dtype=int)
    for i in range(num_labels):
        y_pred[:, i] = (pred_probs[:, i] >= thresholds[i]).astype(int)

    # Báo cáo kết quả
    raw_acc = accuracy_score(y_test.flatten(), y_pred.flatten())
    micro_f1 = f1_score(y_test, y_pred, average='micro', zero_division=0)
    macro_f1 = f1_score(y_test, y_pred, average='macro', zero_division=0)
    weighted_f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
    report = classification_report(y_test, y_pred, target_names=label_list, zero_division=0, digits=4)

    logger.info(f"Raw Accuracy (sklearn, element-wise): {raw_acc:.4f}")
    logger.info(f"Micro-averaged F1 score: {micro_f1:.4f}")
    logger.info(f"Macro-averaged F1 score: {macro_f1:.4f}")
    logger.info(f"Weighted-averaged F1 score: {weighted_f1:.4f}")
    logger.info(f"\nClassification Report (custom thresholds):\n{report}")

    # Phân tích lỗi
    errors = []
    for i, (true, pred, text) in enumerate(zip(y_test, y_pred, test_texts)):
        if not np.array_equal(true, pred):
            errors.append((text, true, pred))
    logger.info(f"Các mẫu dự đoán sai (top 10): {errors[:10]}")

    # Ví dụ kiểm tra mới
    test_examples = [
        "Feeling on top of the world today! 🎉😊 #BestDayEver #SoHappy",
        "Totally let down... 😢💔 #Disappointed #WhyThis",
        "Omg that’s incredible news! 😍✨ #Amazing #Grateful",
        "Head full of thoughts rn 🤯🤔 #Confused #Overthinking",
        "Still can’t believe this happened. So pissed! 😡🔥 #Angry #Unbelievable"
    ]
    results = predict_examples(model, tokenizer, test_examples, label_list, glove_embeddings, emoji2vec, embedding_dim, max_len, thresholds, logger)

    # Lưu kết quả và mô hình
    output_dir = "/content/drive/MyDrive/Goemotions/"
    try:
        os.makedirs(output_dir, exist_ok=True)
        output_file = os.path.join(output_dir, "XGB-results.txt")

        with open(output_file, "a", encoding="utf-8") as f:
            f.write(f"\n=== XGBoost Results - {args.model_type} - {pd.Timestamp.now()} ===\n")
            f.write(f"Raw Accuracy (sklearn, element-wise): {raw_acc:.4f}\n")
            f.write(f"Micro-averaged F1 score: {micro_f1:.4f}\n")
            f.write(f"Macro-averaged F1 score: {macro_f1:.4f}\n")
            f.write(f"Weighted-averaged F1 score: {weighted_f1:.4f}\n")
            f.write(f"\nClassification Report (custom thresholds):\n{report}\n")
            f.write("\nExample Predictions:\n")
            for i, result in enumerate(results):
                f.write(f"Example {i+1}: {result['text']}\n")
                f.write(f"Predicted Labels: {result['labels']}\n")
                f.write(f"Top-3 Labels and Probabilities: {result['top_labels']}\n\n")
            f.write("================================\n")
        logger.info(f"Đã lưu kết quả vào {output_file}")

        # Lưu mô hình
        model_file = os.path.join(args.ckpt_dir, f"{args.model_type}_model.pkl")
        joblib.dump(model, model_file)
        logger.info(f"Đã lưu mô hình vào {model_file}")

        # Lưu ngưỡng
        threshold_file = os.path.join(args.ckpt_dir, f"{args.model_type}_thresholds.pkl")
        with open(threshold_file, 'wb') as f:
            pickle.dump(thresholds, f)
        logger.info(f"Đã lưu ngưỡng vào {threshold_file}")
    except Exception as e:
        logger.error(f"Lỗi khi lưu kết quả hoặc mô hình: {str(e)}")
        raise

if __name__ == "__main__":
    main()